In [1]:
import model_api as dl
import numpy as np
import pandas as pd
import xgboost as xgb

In [2]:
sample_model = dl.NN_classifier(10, 10, verbose=True)
sample_cnn = dl.CNN_classifier(10, 10, verbose=True)
sample_xgboost = dl.XGB_classifier(n_estimators=100)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 10, 3)]           0         
                                                                 
 flatten (Flatten)           (None, 30)                0         
                                                                 
 dense (Dense)               (None, 64)                1984      
                                                                 
 dense_1 (Dense)             (None, 10)                650       
                                                                 
Total params: 2,634
Trainable params: 2,634
Non-trainable params: 0
_________________________________________________________________
Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 10, 3